<style>
  .blue-box {
    background-color: #f0f0f0; /* grey background */
    padding: 10px;
    border: 1px solid #ccc;
    text-align: center;
  }
  .blue-text {
    color: #0000ff; /* blue text */
    font-weight: bold;
  }
</style>

<div class="blue-box">
  <h2 class="blue-text">Forecasting VN1 Data at Monthly Time Granularity with TimesFM</h2>
</div>



### Install Libraries

In [ ]:
!pip install dask[dataframe] protobuf==3.20.* -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 4.4 MB/s eta 0:00:00


In [ ]:
#Install the necessary libraries for forecasting
!pip install statsforecast mlforecast neuralforecast nixtla -q

Importing Libraries Required for this Notebook

In [ ]:
from nixtla import NixtlaClient
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsforecast import StatsForecast
from statsforecast.models import AutoTBATS, AutoARIMA, AutoETS
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from mlforecast import MLForecast
from mlforecast.lag_transforms import (
    RollingMean, RollingStd, RollingMin, RollingMax, RollingQuantile,
    SeasonalRollingMean, SeasonalRollingStd, SeasonalRollingMin,
    SeasonalRollingMax, SeasonalRollingQuantile,ExpandingStd, ExponentiallyWeightedMean,
    ExpandingMean)
from window_ops.expanding import expanding_mean
from window_ops.rolling import rolling_mean
import lightgbm as lgb
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from mlforecast.target_transforms import AutoDifferences
from neuralforecast import NeuralForecast
from neuralforecast.models import TFT, NHITS
from neuralforecast.losses.pytorch import MAE
from neuralforecast.losses.pytorch import MQLoss
from nixtla import NixtlaClient
from functools import reduce
#import timesfm

### Data Preparation

In [ ]:
train = pd.read_csv("VN1_monthly_train.csv")
test = pd.read_csv("VN1_monthly_test.csv")

train['ds'] = pd.to_datetime(train['ds'])
test['ds'] = pd.to_datetime(test['ds'])

In [ ]:
train.head()

,Unnamed: 0,unique_id,ds,y,is_thanksgiving_week,is_christmas_week,Warehouse,Client
0,0,C0_W100_P10705,2020-07-01,28.0,0.0,0.0,W100,C0
1,1,C0_W100_P10705,2020-08-01,35.0,0.0,0.0,W100,C0
2,2,C0_W100_P10705,2020-09-01,28.0,0.0,0.0,W100,C0
3,3,C0_W100_P10705,2020-10-01,28.0,0.0,0.0,W100,C0
4,4,C0_W100_P10705,2020-11-01,35.0,1.0,0.0,W100,C0


In [ ]:
test.head()

,Unnamed: 0,unique_id,ds,is_thanksgiving_week,is_christmas_week,Warehouse,Client
0,0,C0_W100_P10705,2023-10-01,0.0,0.0,W100,C0
1,1,C0_W100_P10705,2023-11-01,1.0,0.0,W100,C0
2,2,C0_W100_P10705,2023-12-01,0.0,1.0,W100,C0
3,4,C0_W100_P10789,2023-10-01,0.0,0.0,W100,C0
4,5,C0_W100_P10789,2023-11-01,1.0,0.0,W100,C0


In [ ]:
train.drop("Unnamed: 0", axis=1, inplace=True)
test.drop("Unnamed: 0", axis=1, inplace=True)

# SFFORECAST

#### Defining AutoARIMA Parameters

In [ ]:
train=train.drop(["Warehouse",	"Client"], axis=1)

In [ ]:
test=test.drop(["Warehouse",	"Client"], axis=1)

In [ ]:
models = [AutoARIMA(season_length=12), AutoTBATS(season_length=12),AutoETS(season_length=12)]

stat_fcst = StatsForecast(models=models,freq='MS')

horizon=3

levels=[5,10,20,30,40,50,60,70,80,90,95]

sf_fcst = stat_fcst.forecast(df=train, h=horizon)

In [ ]:
sf_fcst.to_csv('VN1_sf_monthly.csv', index=False)

### Defining AutoTBATS & AutoETS

In [ ]:
models = [AutoTBATS(season_length=12),AutoETS(season_length=12)]
stat_fcst = StatsForecast(models=models,freq='MS')

In [ ]:
ets_tbats = stat_fcst.forecast(df=train, h=horizon)
ets_tbats.to_csv('VN1_ets_tbats_monthly.csv', index=False)


# MLForecast

In [ ]:
models = [
    lgb.LGBMRegressor(verbosity=-1),
    xgb.XGBRegressor(),
    RandomForestRegressor(random_state=0),
]

mlf = MLForecast(
    models=models,
    lags=[1,2,3,6],
        lag_transforms={
        1: [
            RollingMean(window_size=3),  
            RollingStd(window_size=3),  
            ExpandingMean(), 
            ExpandingStd()
        ],
        3:[
            RollingMean(window_size=3),  
            RollingStd(window_size=3)],
        6: [
            RollingMean(window_size=3),  
            RollingStd(window_size=3),  
            RollingMin(window_size=3),
            RollingMax(window_size=3)]
    },
        date_features=['year', 'month', 'day', 'quarter'],
    freq='MS',
    target_transforms=[AutoDifferences(max_diffs=2)])


In [ ]:
from mlforecast.utils import PredictionIntervals

mlf.fit(
    train2,
    id_col='unique_id',
    time_col='ds',
    target_col='y',
    static_features=[],
    prediction_intervals=PredictionIntervals(n_windows=2, h=3),
)

/usr/local/lib/python3.10/dist-packages/mlforecast/core.py:434: UserWarning: The following series were dropped completely due to the transformations and features: ['C0_W121_P10090', 'C0_W121_P11161', 'C0_W121_P14160', 'C0_W121_P9127', 'C0_W154_P367', 'C0_W154_P655', ...].
These series won't show up if you use `MLForecast.forecast_fitted_values()`.
You can set `dropna=False` or use transformations that require less samples to mitigate this
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/mlforecast/core.py:626: UserWarning: Found null values in lag6, rolling_mean_lag6_window_size3, rolling_std_lag6_window_size3, rolling_min_lag6_window_size3, rolling_max_lag6_window_size3.
  warnings.warn(f'Found null values in {", ".join(cols_with_nulls)}.')
/usr/local/lib/python3.10/dist-packages/mlforecast/core.py:626: UserWarning: Found null values in rolling_mean_lag6_window_size3, rolling_std_lag6_window_size3, rolling_min_lag6_window_size3, rolling_max_lag6_window_size3.
  warnings.warn(f

MLForecast(models=[LGBMRegressor, XGBRegressor, RandomForestRegressor], freq=MS, lag_features=['lag1', 'lag2', 'lag3', 'lag6', 'rolling_mean_lag1_window_size3', 'rolling_std_lag1_window_size3', 'expanding_mean_lag1', 'expanding_std_lag1', 'rolling_mean_lag3_window_size3', 'rolling_std_lag3_window_size3', 'rolling_mean_lag6_window_size3', 'rolling_std_lag6_window_size3', 'rolling_min_lag6_window_size3', 'rolling_max_lag6_window_size3'], date_features=['year', 'month', 'day', 'quarter'], num_threads=1)

In [ ]:
future_df=test.copy()

In [ ]:
levels=[5,10,20,30,40,50,60,70,80,90,95]
mlforecast = mlf.predict(3, X_df=test, level=levels)

In [ ]:
mlforecast.head()

,unique_id,ds,LGBMRegressor,XGBRegressor,RandomForestRegressor,LGBMRegressor-lo-95,LGBMRegressor-lo-90,LGBMRegressor-lo-80,LGBMRegressor-lo-70,LGBMRegressor-lo-60,...,RandomForestRegressor-hi-10,RandomForestRegressor-hi-20,RandomForestRegressor-hi-30,RandomForestRegressor-hi-40,RandomForestRegressor-hi-50,RandomForestRegressor-hi-60,RandomForestRegressor-hi-70,RandomForestRegressor-hi-80,RandomForestRegressor-hi-90,RandomForestRegressor-hi-95
0,C0_W100_P10705,2023-10-01,14.610540,11.844671,9.80,3.847005,4.560074,5.986211,7.412349,8.838486,...,10.688,11.576,12.464,13.046,13.4750,13.904,14.3330,14.762,15.1910,15.40550
1,C0_W100_P10705,2023-11-01,29.051099,28.923181,11.19,18.386127,18.964402,20.120954,21.277505,22.434056,...,12.192,13.194,14.196,15.274,16.3900,17.506,18.6220,19.738,20.8540,21.41200
2,C0_W100_P10705,2023-12-01,33.661639,26.521893,10.04,14.792103,15.486453,16.875151,18.263849,19.652548,...,11.210,12.380,13.550,14.159,14.4875,14.816,15.1445,15.473,15.8015,15.96575
3,C0_W100_P10789,2023-10-01,14.608710,15.788490,19.75,6.172074,6.212042,6.291978,6.371914,6.451850,...,20.953,22.156,23.359,24.598,25.8550,27.112,28.3690,29.626,30.8830,31.51150
4,C0_W100_P10789,2023-11-01,29.049269,34.510406,12.57,6.661090,6.979016,7.614867,8.250719,8.886571,...,16.800,21.030,25.260,27.541,28.8475,30.154,31.4605,32.767,34.0735,34.72675


In [ ]:
mlforecast.to_csv('mlforecast_VN1_Monthly.csv', index=False)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Neural Forecast

In [ ]:
hist_exog_list = ["is_thanksgiving_week",	"is_christmas_week"]

horizon = len(test)  
tft_model = TFT(h=3,  
                input_size=8, 
                hist_exog_list=hist_exog_list,
                max_steps=5000,
                start_padding_enabled=True,
                loss=MQLoss(level=levels))


nhits_model = NHITS(h=3,  
                input_size=8,
                hist_exog_list=hist_exog_list,
                max_steps=5000,
                start_padding_enabled=True,
                loss=MQLoss(level=levels))

INFO:lightning_fabric.utilities.seed:Seed set to 1
INFO:lightning_fabric.utilities.seed:Seed set to 1


In [ ]:
nf = NeuralForecast(models=[tft_model, nhits_model], freq='MS')  # 'M' for monthly frequency

nf.fit(df=train)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                    | Type                     | Params | Mode 
-----------------------------------------------------------------------------
0 | loss                    | MQLoss                   | 23     | train
1 | padder_train            | ConstantPad1d            | 0      | train
2 | scaler                  | TemporalNorm             | 0      | train
3 | embedding               | TFTEmbedding             | 1.0 K  | train
4 | temporal_encoder        | TemporalCovariateEncoder | 780 K  | train
5 | temporal_fusion_decoder | TemporalFusionDecoder    | 256 K  | train
6 | output_adapter          | Linear      

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=5000` reached.


In [ ]:
neural_fcst = nf.predict(futr_df = test)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


In [ ]:
neural_fcst.reset_index(inplace=True)

In [ ]:
neural_fcst

,unique_id,ds,TFT-median,TFT-lo-95,TFT-lo-90,TFT-lo-80,TFT-lo-70,TFT-lo-60,TFT-lo-50,TFT-lo-40,...,NHITS-hi-10,NHITS-hi-20,NHITS-hi-30,NHITS-hi-40,NHITS-hi-50,NHITS-hi-60,NHITS-hi-70,NHITS-hi-80,NHITS-hi-90,NHITS-hi-95
0,C0_W100_P10705,2023-10-01,9.746429,-3.273524,-0.526658,2.509175,4.335842,5.522599,6.443741,7.225574,...,10.578292,11.405527,12.160178,13.249665,14.562010,15.972660,17.957291,20.140085,25.862354,34.493343
1,C0_W100_P10705,2023-11-01,9.109008,-4.439062,-1.523023,1.665268,3.500026,4.619958,5.597836,6.431686,...,10.876289,11.665981,12.817421,13.900038,15.614113,17.235762,19.466400,23.035103,29.899458,40.687702
2,C0_W100_P10705,2023-12-01,11.122757,-3.216594,-0.503346,2.543634,4.474517,5.702237,6.750391,7.767630,...,10.853306,11.845634,12.688111,14.348911,15.917175,18.571032,21.666946,24.996552,32.767181,42.746521
3,C0_W100_P10789,2023-10-01,15.094298,-29.003628,-18.980045,-8.712521,-3.364260,0.187744,3.506044,5.722885,...,17.624611,21.772297,25.144341,29.010731,35.498711,42.521385,48.855362,61.093063,84.723846,102.745529
4,C0_W100_P10789,2023-11-01,17.132938,-31.311462,-20.362633,-9.404289,-3.628897,0.002804,3.612181,6.099564,...,16.034801,20.068790,24.121605,29.052246,34.922607,40.542366,48.028824,60.785530,88.594391,122.467377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17395,C8_W276_P8678,2023-11-01,7.631420,-0.693712,1.300267,3.261192,4.305410,4.925702,5.492183,5.981553,...,8.217737,8.820065,9.712059,10.976864,12.365852,14.275718,17.004869,20.704067,28.070648,37.545044
17396,C8_W276_P8678,2023-12-01,8.126240,-0.655031,1.342755,3.318986,4.412111,5.056252,5.663147,6.224548,...,8.485082,9.520420,10.565582,11.947353,13.855113,16.262772,18.823648,23.773647,30.768354,41.500240
17397,C8_W276_P9629,2023-10-01,49.249531,18.803883,25.514656,32.524025,36.285244,39.104965,41.199028,42.996109,...,55.016533,56.525902,60.099716,62.056175,65.561111,70.392830,75.107178,82.354683,95.988373,108.473221
17398,C8_W276_P9629,2023-11-01,44.794090,9.379654,18.045769,26.421522,30.787975,33.759945,36.278385,38.236916,...,53.665565,55.029068,57.807892,61.263672,63.852783,67.858467,71.837540,80.410683,94.782188,111.245499


In [ ]:
neural_fcst.to_csv('neural_fcst_VN1_monthly.csv', index=False)


# TimeGPT

In [ ]:

nixtla_client = NixtlaClient(api_key='your key')

# Assuming df is your input dataframe with historical data
# and future_df is your dataframe with future exogenous variables

# Specify the exogenous variables
exog_vars = ["is_thanksgiving_week",	"is_christmas_week"]
# Make the forecast
timegpt_fcst_zero = nixtla_client.forecast(df=train, h=3, X_df=test, freq='MS')


In [ ]:
timegpt_fcst_zero['ds'] = pd.to_datetime(timegpt_fcst_zero['ds'])

In [ ]:
timegpt_fcst_zero.to_csv("timegpt_monthly.csv")

## TimesFM

In [ ]:
#Reading the forecasted file
timesfm_df = pd.read_csv("timesfm_monthly_VN1.csv")
timesfm_df['ds'] = pd.to_datetime(timesfm_df['ds'])

### Merge all forecasts

In [ ]:
data_frames = [sf_fcst, ets_tbats,mlforecast, neural_fcst,timegpt_fcst_zero,timesfm_df ]
final_forecast_monthly_merged = reduce(lambda  left,right: pd.merge(left,right,on=['unique_id', 'ds'],how='inner'), data_frames)
final_forecast_monthly_merged.to_csv("final_forecast_monthly_merged.csv")